In [257]:
from bs4 import BeautifulSoup
import urllib.request
import re

#write collected data to .csv file as a pandas dataframe
import numpy
import pandas as pd 

#decode geotagging answer
import json
import csv

#extract price
def price_from_json(flat):
    price = flat.find("div", {"class": "popup-prices"})['data-prices']
    price_ind = price.find('RUB":')
    price_cut = price[price_ind + 5:]
    price_in_rub = price_cut.split(",")[0]
    return int(price_in_rub)

In [ ]:
#url to first pages

cian_url = "https://spb.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=2&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1"
url = "https://www.avito.ru/sankt-peterburg/kvartiry?s_trg=1"

In [ ]:

avito_real_estate = urllib.request.urlopen("https://www.avito.ru/sankt-peterburg/kvartiry/prodam").read()
#parse avito with beautiful soup
avito_soup = BeautifulSoup(avito_real_estate)
url_base = "https://www.avito.ru"
flats = avito_soup.findAll("div", {"class":"item_table-description"})
page_count = 0

data_flats = parse_page(flats, [])

while True:
    try:
        page_count += 1
        print("Page: ", page_count)
        next_page = avito_soup.find("a", {"class":"pagination-page"}).get('href')
        flats = avito_soup.findAll("div", {"class":"item_table-description"})
        data_flats = parse_page_avito(flats, data_flats)
        
        if page_count > 100:
                break
    except Error:
        break

print("finish")
print("data len: ", len(data_flats))



In [230]:
#extract all relevant data from single page
def parse_page_avito(flats, data_flats):
    rent_count = 0
    print("flats on page: ", len(flats))
    for (index, flat) in enumerate(flats):

        print("Flat number", index)

        #price
        print("price:")
        price = price_from_json(flat)
        if(price < 200_000):
            rent_count += 1
            print("Rent found, skip")
            continue
        print(price)
        print()


        #rooms, area, floor level
        print("rooms:")
        description = flat.find("a", {"class": "item-description-title-link"})['title']
        try_rooms = description.split(",")[0][0]
        if try_rooms.isnumeric():
            rooms = int(try_rooms)
        else:
            rooms = 1
        print(rooms)
        print()

        print("area:")
        area = float(description.split(",")[1].split(" ")[1])
        print(area, "m^2")
        print()

        print(description)

        print("floor:")
        floor = description.split(",")[2].split(" ")[1].split("/")[0]
        floors_total = description.split(",")[2].split(" ")[1].split("/")[1]
        print("" + floor + ", total floors: " + floors_total)
        print()

        #subway, distance to subway, adress
        full_adress = flat.find("p", {"class": "address"}).getText()

        adress = ','.join(full_adress.split(",")[1:])
        print("adress:")
        print(adress)
        print()

        subway = full_adress.split(",")[0]
        for (ind,el) in enumerate(subway.split()):
            try:
                float(el)
                subway_station = ' '.join(subway.split()[:ind])
                distance = float(subway.split()[ind])
            except ValueError:
                pass

        print("subway:")
        print(subway_station)
        print()

        if distance < 99:
            distance *= 1000
        print("distance to subway:")
        print(distance, "meters")
        print()

        print("go to next page")
        data_flats.append((price, rooms, area, floor, floors_total, adress, subway_station, distance))
    
    print(rent_count)
    return data_flats
    

In [202]:
#flat data class combining all relevant data
class Flat:
    
    def __init__(self, flat_price, flat_rooms, flat_area, flat_floor, 
                 flat_floors_total, flat_address, flat_subway_st, flat_distance_to_subway):
        self.flat_price = flat_price
        self.flat_rooms = flat_rooms 
        self.flat_area = flat_area
        self.flat_floor = flat_floor
        self.flat_floors_total = flat_floors_total
        self.flat_address = flat_address
        self.flat_subway_st = flat_subway_st
        self.flat_distance_to_subway = flat_distance_to_subway
        
    def __str__(self):
        return ''.join("adress = "+ flat_address + 
                       ", price = "+ str(flat_price)+ 
                       ", rooms = "+ str(flat_rooms)+
                        ", floor = "+ str(flat_floor)+ 
                       ", area = " + str(flat_area) +
                      ", total_floors = "+ str(flat_floors_total)+ 
                       ", subway = "+ flat_subway_st + 
                       ", distance to subway = " + flat_distance_to_subway)

In [220]:
#
subways = set()
for flat in data_flats:
    subways.add(flat[6])
print(subways)

{'Чкаловская', 'Проспект Просвещения', 'Пионерская', 'Пролетарская', 'Приморская', 'Купчино', 'Проспект Большевиков', 'Елизаровская', 'Ломоносовская', 'Комендантский проспект', 'Василеостровская', 'Ладожская', 'Девяткино', 'Спортивная', 'Московская', 'Чернышевская', 'Автово', 'Звездная', 'Обухово', 'Парнас', 'Достоевская', 'Проспект Ветеранов', 'Новочеркасская', 'Гражданский проспект'}


In [225]:
#clean up parsed data
new_data_flats = []
for flat in data_flats:
    new_flat = (flat[0],flat[1],flat[2],flat[3],flat[4],flat[5].replace('\xa0', '', 2),flat[6],flat[7])
    new_data_flats.append(new_flat)

In [229]:
#write collected data to .csv file as a pandas dataframe
import numpy
import pandas as pd 

a = numpy.asarray(new_data_flats)
pd.DataFrame(a).to_csv("data.csv", header=['price', 'rooms', 'area', 'floor', 'floors_total', 'adress',
                                           'subway_station', 'distance to subway'])

 Парс сайта cian.ru

In [154]:
#read page, load to soup object
cian_real_estate = urllib.request.urlopen("https://spb.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=2&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1").read()
cian_soup = BeautifulSoup(cian_real_estate)

In [258]:
def next_page(first_page, soup):
    next_p = soup.find("li", {"class": re.compile('^.*list-item--active.*')})
    if next_p and next_p.nextSibling:
        return next_p.nextSibling.a['href']
    else:
        return -1

def cian_flats(cian_soup, data_flats):
    counter = 0
    flats = cian_soup.findAll("div", {"class": "c6e8ba5398--info-container--YhZ2y"})
    print('flats = ',len(flats))
    for flat in flats:
        
        counter += 1
        # rooms, area, floor
        addr = flat.find("div", {"class": re.compile('^.*title.*')})
        if addr:
            try:
                rooms = int(addr.text[0])
                print('rooms = ', rooms)
            except Exception:
                rooms = 1
            
            area = float(addr.text.split(',')[1].split()[0])
            print('area = ', area)
            floor = int(addr.text.split(',')[2].split(' ')[1].split('/')[0])
            print('floors = ', floor)
            floors_total = int(addr.text.split(',')[2].split(' ')[1].split('/')[1])
            print('floors total = ', floors_total)
            
            print(addr.text)

        subway = flat.find("div", {"class": re.compile('^.*underground-name.*')})
        if subway:
            subway = subway.text
            print(subway)
        else:
            subway = None

        remoteness = flat.find("div", {"class": re.compile('^.*remoteness.*')})
        if remoteness:
            print(remoteness.text)
            if remoteness.text.split('\xa0')[2] == 'пешком':
                remoteness = int(remoteness.text.split('\xa0')[0]) * 100
            else:
                remoteness = int(remoteness.text.split('\xa0')[0]) * 1000
            print('distance to subway = ',remoteness)
        else:
            remoteness = None
            
        adress = flat.findAll("a", {"class": re.compile('^.*address.*')})
        if adress:
            if(len(adress) > 5):
                adress = adress[4].text+ ', ' + adress[5].text
                print(adress)
            else:
                continue
                
        price = flat.find("div", {"class": re.compile('^.*price.*')}).find("div",{'class': re.compile('.*header.*')})
        if price:
            print(price.text)
            price_array = price.text.split(' ')[:len(price.text.split(' ')) - 1]
            price = 0
            for el in price_array:
                price *= 1000
                price += int(el)
            print('price = ', price)
            
        data_flats.append((price, rooms, area, floor, floors_total, adress, subway, remoteness))
        print()
    return data_flats

In [260]:
cian_data

[(11945000, 1, 51.0, 6, 8, 'Смольный проспект, 17', 'Чернышевская', 1000),
 (18960000, 3, 181.0, 10, 10, 'Афанасьевская улица, 1', 'Озерки', 5000),
 (10495000,
  3,
  86.0,
  10,
  11,
  'Кременчугская улица, 17к3',
  'Площадь Восстания',
  1000),
 (5650000,
  1,
  37.0,
  14,
  17,
  'Богатырский проспект, 36к1',
  'Комендантский проспект',
  1800),
 (7650000, 3, 75.0, 1, 5, 'Перевозный переулок, 9', 'Новочеркасская', 500),
 (5750000,
  1,
  43.0,
  17,
  20,
  'Парашютная улица, 38к4',
  'Комендантский проспект',
  3000),
 (18970000, 4, 256.0, 10, 10, 'Афанасьевская улица, 1', 'Удельная', 8000),
 (22750000, 5, 420.0, 10, 10, 'Афанасьевская улица, 1', 'Озерки', 2600),
 (6499000,
  1,
  43.0,
  10,
  10,
  'Приморский проспект, 54к2',
  'Старая Деревня',
  3100),
 (7730000, 3, 84.0, 1, 4, 'Пулковское тер., Неоклассика ЖК', 'Купчино', None),
 (9022000, 2, 65.0, 13, 19, 'Общественный переулок, 5', 'Елизаровская', 1000),
 (17000000, 3, 86.0, 7, 11, 'Пионерская улица, 50', 'Чкаловская', 15

In [264]:
cian_url = "https://spb.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=2&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1"
base_url = cian_url.split('cian.ru')[0] + 'cian.ru' 
cian_real_estate = urllib.request.urlopen(cian_url).read()
cian_soup = BeautifulSoup(cian_real_estate)
cian_data = cian_flats(cian_soup, [])

28
rooms =  1
area =  37.0
floors =  11
floors total =  16
1-комн. кв., 37 м², 11/16 этаж
Старая Деревня
20 минут пешком
distance to subway =  2000
Стародеревенская улица, 18
5 500 000 ₽
price =  5500000

rooms =  1
area =  55.0
floors =  4
floors total =  14
1-комн. кв., 55 м², 4/14 этаж
Площадь Ленина
10 минут на транспорте
distance to subway =  10000
улица Жукова, 1
7 547 000 ₽
price =  7547000

rooms =  2
area =  58.0
floors =  18
floors total =  25
2-комн. кв., 58 м², 18/25 этаж
Комендантский проспект
30 минут пешком
distance to subway =  3000
rooms =  3
area =  181.0
floors =  10
floors total =  10
3-комн. кв., 181 м², 10/10 этаж
Озерки
5 минут на транспорте
distance to subway =  5000
Афанасьевская улица, 1
18 960 000 ₽
price =  18960000

rooms =  3
area =  86.0
floors =  10
floors total =  11
3-комн. кв., 86 м², 10/11 этаж
Площадь Восстания
10 минут пешком
distance to subway =  1000
Кременчугская улица, 17к3
10 495 000 ₽
price =  10495000

rooms =  1
area =  37.0
floors =  14
fl

In [266]:
base_url = cian_url.split('cian.ru')[0] + 'cian.ru' 
cian_real_estate = urllib.request.urlopen(cian_url).read()
cian_soup = BeautifulSoup(cian_real_estate)
counter = 0
flats = []
while counter < 59:
    counter += 1
    flats = cian_flats(cian_soup, flats)
    cian_url = next_page(cian_url, cian_soup)
    if cian_url == -1:
        break
    if base_url not in cian_url:
        cian_url = base_url + cian_url
    print("url = ", cian_url)
    cian_html = urllib.request.urlopen(cian_url).read()
    cian_soup = BeautifulSoup(cian_html)
    print("counter = ", counter)
    
    
print('finished parsing, writing csv')
a = numpy.asarray(flats)
pd.DataFrame(a).to_csv("cian_data.csv", header=['price', 'rooms', 'area', 'floor', 'floors_total', 'adress',
                                           'subway_station', 'distance to subway'])

28
area =  72.0
floors =  5
floors total =  6
Европейская планировка3-комн. кв., 72 м², 5/6 этаж
Озерки
12 минут на транспорте
distance to subway =  12000
1-я Утиная улица, 28
9 000 000 ₽
price =  9000000

rooms =  2
area =  66.0
floors =  2
floors total =  7
2-комн. кв., 66 м², 2/7 этаж
Чернышевская
rooms =  2
area =  94.0
floors =  6
floors total =  8
2-комн. кв., 94 м², 6/8 этаж
Площадь Восстания
8 минут пешком
distance to subway =  800
4-я Советская улица, 9
22 080 000 ₽
price =  22080000

rooms =  3
area =  181.0
floors =  10
floors total =  10
3-комн. кв., 181 м², 10/10 этаж
Озерки
5 минут на транспорте
distance to subway =  5000
Афанасьевская улица, 1
18 960 000 ₽
price =  18960000

rooms =  3
area =  86.0
floors =  10
floors total =  11
3-комн. кв., 86 м², 10/11 этаж
Площадь Восстания
10 минут пешком
distance to subway =  1000
Кременчугская улица, 17к3
10 495 000 ₽
price =  10495000

rooms =  1
area =  37.0
floors =  14
floors total =  17
1-комн. кв., 37 м², 14/17 этаж
Комендан

counter =  2
28
rooms =  5
area =  239.0
floors =  5
floors total =  5
5-комн. кв., 239 м², 5/5 этаж
Невский проспект
10 минут пешком
distance to subway =  1000
Большая Конюшенная улица, 13
105 000 000 ₽
price =  105000000

rooms =  1
area =  37.0
floors =  3
floors total =  25
1-комн. кв., 37 м², 3/25 этаж
Комендантский проспект
30 минут пешком
distance to subway =  3000
area =  28.0
floors =  4
floors total =  23
Студия, 28 м², 4/23 этаж
Ломоносовская
15 минут на транспорте
distance to subway =  15000
улица Русановская, 19к2
2 720 000 ₽
price =  2720000

rooms =  4
area =  87.0
floors =  1
floors total =  9
4-комн. кв., 87 м², 1/9 этаж
Гражданский проспект
16 минут пешком
distance to subway =  1600
улица Брянцева, 2К1
7 450 000 ₽
price =  7450000

rooms =  3
area =  117.0
floors =  9
floors total =  12
3-комн. кв., 117 м², 9/12 этаж
Василеостровская
10 минут на транспорте
distance to subway =  10000
Уральская улица, 6
16 062 760 ₽
price =  16062760

rooms =  3
area =  150.0
floors = 

counter =  4
28
rooms =  3
area =  141.0
floors =  9
floors total =  11
3-комн. кв., 141 м², 9/11 этаж
Петроградская
15 минут пешком
distance to subway =  1500
улица Грота, 1-3В
40 554 000 ₽
price =  40554000

rooms =  1
area =  48.0
floors =  8
floors total =  11
1-комн. кв., 48 м², 8/11 этаж
Василеостровская
20 минут пешком
distance to subway =  2000
rooms =  4
area =  151.0
floors =  1
floors total =  4
4-комн. кв., 151 м², 1/4 этаж
rooms =  3
area =  134.0
floors =  8
floors total =  20
3-комн. кв., 134 м², 8/20 этаж
Площадь Мужества
15 минут пешком
distance to subway =  1500
rooms =  2
area =  124.0
floors =  6
floors total =  9
2-комн. кв., 124 м², 6/9 этаж
Чернышевская
15 минут пешком
distance to subway =  1500
набережная Воскресенская, 30
68 238 500 ₽
price =  68238500

rooms =  1
area =  32.0
floors =  4
floors total =  12
1-комн. кв., 32 м², 4/12 этаж
Ладожская
7 минут на транспорте
distance to subway =  7000
Индустриальный проспект, 27
3 879 230 ₽
price =  3879230

rooms =  

counter =  6
28
rooms =  4
area =  148.0
floors =  6
floors total =  9
4-комн. кв., 148 м², 6/9 этаж
Выборгская
2 минуты пешком
distance to subway =  200
Крапивный переулок, 4
27 407 570 ₽
price =  27407570

rooms =  2
area =  65.0
floors =  20
floors total =  25
2-комн. кв., 65 м², 20/25 этаж
Комендантский проспект
30 минут пешком
distance to subway =  3000
rooms =  3
area =  102.0
floors =  23
floors total =  24
3-комн. кв., 102 м², 23/24 этаж
Комендантский проспект
9 минут на транспорте
distance to subway =  9000
Парашютная улица, 52
10 800 000 ₽
price =  10800000

rooms =  4
area =  112.0
floors =  22
floors total =  24
4-комн. кв., 112 м², 22/24 этаж
Ленинский проспект
10 минут на транспорте
distance to subway =  10000
бульвар Балтийский, 4
14 229 000 ₽
price =  14229000

rooms =  2
area =  87.0
floors =  3
floors total =  8
2-комн. кв., 87 м², 3/8 этаж
Горьковская
10 минут пешком
distance to subway =  1000
улица Мира, 36
14 272 500 ₽
price =  14272500

rooms =  3
area =  102.0
fl

counter =  8
28
area =  122.0
floors =  3
floors total =  7
Квартира в центре С-Петербурга3-комн. кв., 122 м², 3/7 этаж
Владимирская
7 минут пешком
distance to subway =  700
Загородный проспект, 24
10 990 000 ₽
price =  10990000

area =  203.0
floors =  4
floors total =  10
Видовая квартира на Малую Невку!2-комн. кв., 203 м², 4/10 этаж
Чкаловская
16 минут пешком
distance to subway =  1600
Ждановская улица, 45
59 500 000 ₽
price =  59500000

rooms =  2
area =  85.0
floors =  6
floors total =  9
2-комн. кв., 85 м², 6/9 этаж
Выборгская
2 минуты пешком
distance to subway =  200
Крапивный переулок, 4
15 429 180 ₽
price =  15429180

rooms =  1
area =  42.0
floors =  12
floors total =  26
1-комн. кв., 42 м², 12/26 этаж
Парнас
6 минут пешком
distance to subway =  600
улица Михаила Дудина, 25к2
4 300 000 ₽
price =  4300000

area =  30.0
floors =  14
floors total =  14
Студия, 30 м², 14/14 этаж
Фрунзенская
10 минут пешком
distance to subway =  1000
улица Парфеновская, 9к2
4 601 380 ₽
price =  46

...

counter =  50
28
rooms =  1
area =  38.0
floors =  16
floors total =  25
1-комн. кв., 38 м², 16/25 этаж
Комендантский проспект
30 минут пешком
distance to subway =  3000
rooms =  2
area =  81.0
floors =  7
floors total =  8
2-комн. кв., 81 м², 7/8 этаж
Чернышевская
15 минут пешком
distance to subway =  1500
Орловская улица, 1к2
35 000 000 ₽
price =  35000000

rooms =  3
area =  180.0
floors =  14
floors total =  14
3-комн. кв., 180 м², 14/14 этаж
Площадь Ленина
10 минут на транспорте
distance to subway =  10000
улица Жукова, 3
26 900 000 ₽
price =  26900000

rooms =  2
area =  36.0
floors =  13
floors total =  19
2-комн. кв., 36 м², 13/19 этаж
Ленинский проспект
7 минут на транспорте
distance to subway =  7000
area =  25.0
floors =  25
floors total =  26
Студия, 25 м², 25/26 этаж
Старая Деревня
10 минут на транспорте
distance to subway =  10000
Яхтенная улица, 28
3 950 010 ₽
price =  3950010

rooms =  1
area =  43.0
floors =  1
floors total =  12
1-комн. кв., 43 м², 1/12 этаж
Проспект 

counter =  52
28
rooms =  2
area =  65.0
floors =  6
floors total =  8
2-комн. кв., 65 м², 6/8 этаж
Чкаловская
5 минут пешком
distance to subway =  500
rooms =  1
area =  42.0
floors =  14
floors total =  25
1-комн. кв., 42 м², 14/25 этаж
Звёздная
10 минут пешком
distance to subway =  1000
Пулковская улица, 6к5
5 283 770 ₽
price =  5283770

rooms =  2
area =  45.0
floors =  1
floors total =  5
2-комн. кв., 45 м², 1/5 этаж
Электросила
15 минут пешком
distance to subway =  1500
Благодатная улица, 15
4 089 000 ₽
price =  4089000

rooms =  1
area =  39.0
floors =  3
floors total =  16
1-комн. кв., 39 м², 3/16 этаж
Площадь Ленина
10 минут на транспорте
distance to subway =  10000
проспект Металлистов, 117
4 032 000 ₽
price =  4032000

area =  31.0
floors =  6
floors total =  12
Апартаменты-студия, 31 м², 6/12 этаж
Приморская
10 минут пешком
distance to subway =  1000
rooms =  1
area =  35.0
floors =  14
floors total =  20
1-комн. кв., 35 м², 14/20 этаж
Ленинский проспект
10 минут на транспо

counter =  54
28
rooms =  2
area =  57.0
floors =  5
floors total =  5
2-комн. кв., 57 м², 5/5 этаж
Звёздная
15 минут на транспорте
distance to subway =  15000
Оранжерейная улица, 51а
6 500 000 ₽
price =  6500000

rooms =  1
area =  37.0
floors =  20
floors total =  25
1-комн. кв., 37 м², 20/25 этаж
Комендантский проспект
30 минут пешком
distance to subway =  3000
rooms =  3
area =  127.0
floors =  2
floors total =  6
3-комн. кв., 127 м², 2/6 этаж
Московская
11 минут пешком
distance to subway =  1100
Московский проспект, 208
33 000 000 ₽
price =  33000000

rooms =  1
area =  38.0
floors =  13
floors total =  25
1-комн. кв., 38 м², 13/25 этаж
Старая Деревня
10 минут на транспорте
distance to subway =  10000
rooms =  1
area =  39.0
floors =  9
floors total =  26
1-комн. кв., 39 м², 9/26 этаж
Парнас
15 минут пешком
distance to subway =  1500
Заречная улица, 11к4
4 050 000 ₽
price =  4050000

rooms =  2
area =  36.0
floors =  6
floors total =  12
2-комн. кв., 36 м², 6/12 этаж
Московская
7 

counter =  56
28
area =  50.0
floors =  23
floors total =  23
Супер квартира. All inclusive.2-комн. кв., 50 м², 23/23 этаж
Ломоносовская
15 минут на транспорте
distance to subway =  15000
улица Русановская, 17к3
5 600 000 ₽
price =  5600000

rooms =  5
area =  188.0
floors =  5
floors total =  6
5-комн. кв., 188 м², 5/6 этаж
Владимирская
6 минут пешком
distance to subway =  600
улица Рубинштейна, 15-17
38 900 000 ₽
price =  38900000

area =  32.0
floors =  3
floors total =  17
Студия, 32 м², 3/17 этаж
Лесная
15 минут пешком
distance to subway =  1500
Большой Сампсониевский проспект, 77/7
4 054 420 ₽
price =  4054420

rooms =  1
area =  43.0
floors =  1
floors total =  10
1-комн. кв., 43 м², 1/10 этаж
Чёрная речка
30 минут на транспорте
distance to subway =  30000
улица Гагаринская, 77к1
4 186 560 ₽
price =  4186560

rooms =  1
area =  40.0
floors =  7
floors total =  21
1-комн. кв., 40 м², 7/21 этаж
Старая Деревня
20 минут пешком
distance to subway =  2000
Полевая Сабировская улица, 45

counter =  58
28
rooms =  1
area =  38.0
floors =  18
floors total =  25
1-комн. кв., 38 м², 18/25 этаж
Комендантский проспект
30 минут пешком
distance to subway =  3000
area =  19.0
floors =  2
floors total =  5
Квартира-студия на МаяковскогоСтудия, 19 м², 2/5 этаж
Маяковская
10 минут пешком
distance to subway =  1000
улица Маяковского, 19/15
2 688 000 ₽
price =  2688000

rooms =  4
area =  118.0
floors =  3
floors total =  8
4-комн. кв., 118 м², 3/8 этаж
Крестовский остров
12 минут пешком
distance to subway =  1200
area =  35.0
floors =  4
floors total =  12
Студия, 35 м², 4/12 этаж
Комендантский проспект
10 минут на транспорте
distance to subway =  10000
rooms =  1
area =  35.0
floors =  11
floors total =  11
1-комн. кв., 35 м², 11/11 этаж
Площадь Мужества
15 минут пешком
distance to subway =  1500
Политехническая улица, 4А
4 225 770 ₽
price =  4225770

rooms =  2
area =  45.0
floors =  1
floors total =  5
2-комн. кв., 45 м², 1/5 этаж
Международная
7 минут на транспорте
distance to 

In [277]:
import requests
api_key = '800e1c1a-3d68-496e-88a9-009b98d7eb75'
city = "Санкт-Петербург"
r = requests.get('https://geocode-maps.yandex.ru/1.x/?apikey=800e1c1a-3d68-496e-88a9-009b98d7eb75&format=json&geocode=Санкт-Петербург,+улица+Жукова,+1')

print(r.json)

<bound method Response.json of <Response [200]>>


In [297]:

d = json.loads(r.text)
longitude, latitude = d['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split()

('30.405062', '59.962435')

In [307]:

#add geotagging to existing csv file
with open('cian_data copy.csv') as csv_file, open('cian_write.csv', 'w') as out:
    
    writer = csv.writer(out, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        writer.writerow((*row, 'longitude', 'latitude'))
        